In [7]:
def modularity_fitness(repres, problParam):
    '''
    Fitness function for the modularity maximization problem.
    :param repres:
    :param problParam:
    :return:
    '''
    graph = problParam['graph']
    min_node = min(graph.nodes)
    communities = decode_communities(repres, problParam['numNodes'], min_node)

    community_list = list(communities.values())
    # print(f"Communities: {communities}")
    # print(f"Community List: {community_list}")

    if not is_partition(graph, community_list):
        raise ValueError(f"{community_list} is not a valid partition of the graph")

    return -modularity(graph, community_list)

In [8]:
def my_modularity_fitness(repres, graph):
    communities = decode_communities(repres, graph.number_of_nodes())
    m = graph.number_of_edges()
    q = 0
    for community in communities:
        for i in community:
            for j in community:
                if i != j:
                    if graph.has_edge(i, j):
                        Aij = 1
                    else:
                        Aij = 0
                    ki = graph.degree(i)
                    kj = graph.degree(j)
                    q += Aij - (ki * kj) / (2 * m)
    return q / (2 * m)

In [9]:
def inter_community_edges_fitness(repres, problParam):
    fitness = 0
    graph = problParam['graph']
    communities = decode_communities(repres, problParam['numNodes'])
    for u, v in graph.edges():
        if u not in communities or v not in communities:
            raise ValueError(f"Nodurile {u} sau {v} nu sunt prezente în comunități.")
        if communities[u] != communities[v]:
            fitness += 1
    return fitness

In [10]:
import networkx as nx


# Edge Density within Communities
def edge_density_fitness(repres, graph):
    communities = decode_communities(repres)
    density = 0
    for community in communities:
        subgraph = graph.subgraph(community)
        density += nx.density(subgraph)
    return -density


In [11]:
def my_edge_density_fitness(repres, graph):
    communities = decode_communities(repres)
    density = 0
    for community in communities:
        subgraph = graph.subgraph(community)
        m = subgraph.number_of_edges()
        n = subgraph.number_of_nodes()
        density += m / (n * (n - 1) / 2)
    return -density

In [12]:
def decode_communities(repres, num_nodes, min_node=0):
    '''
    Decodes a chromosome into a partition of the graph nodes into communities.
    :param repres:
    :param num_nodes:
    :return:
    '''
    if len(repres) != num_nodes:
        raise ValueError("Lungimea reprezentării nu corespunde numărului de noduri.")

    communities = {i: [] for i in range(max(repres) + 1)}
    for node, community in enumerate(repres):
        communities[community].append(node + min_node)

    # Check if all nodes are included
    all_nodes = set(range(min_node, min_node + num_nodes))
    included_nodes = set(node for community in communities.values() for node in community)

    if all_nodes != included_nodes:
        missing_nodes = all_nodes - included_nodes
        raise ValueError(f"Missing nodes in communities: {missing_nodes}")

    # Remove empty communities
    communities = {k: v for k, v in communities.items() if v}

    return communities